In [ ]:
!pip install -q google-adk

In [ ]:
import os
try:
    from google.colab import userdata
    os.environ["GOOGLE_API_KEY"] = userdata.get("GOOGLE_API_KEY")
except ImportError:
    pass  # Not running in Colab; uses env var already set

# Study Buddy Agent

Build an agent that helps students learn. Your agent will have these tools:

1. **create_flashcard** — Create a flashcard with a question and answer
2. **quiz_me** — Generate a quiz question on a given topic
3. **explain_concept** — Provide a simplified explanation of a concept at a chosen difficulty level

Tools return hardcoded/fake data — the goal is designing the **interface** between the model and your code.

## Setup

In [ ]:
from google.adk.agents import Agent
from google.adk.runners import InMemoryRunner
from google.genai import types

## TODO 1: Define your tools

Just write plain Python functions. The ADK infers the schema from:
- **Type hints** → parameter types
- **Docstring** → tool description

The first tool is done for you as an example. Implement the remaining two.

In [ ]:
# Example tool (complete)

def create_flashcard(topic: str, question: str, answer: str) -> dict:
    """Create a flashcard for studying.

    Args:
        topic: The subject area (e.g. "Biology", "History").
        question: The front of the flashcard.
        answer: The back of the flashcard.

    Returns:
        dict: status and confirmation or error message.
    """
    return {
        "status": "success",
        "report": (
            f"Flashcard created for {topic}! "
            f"Q: {question} | A: {answer}"
        ),
    }


def quiz_me(topic: str, difficulty: str) -> dict:
    """Generate a quiz question on a topic.

    Args:
        topic: The subject to quiz on (e.g. "Photosynthesis", "World War II").
        difficulty: Question difficulty — "easy", "medium", or "hard".

    Returns:
        dict: status and a quiz question with answer or error message.
    """
    # TODO: implement — return a dict with "status" and "report" keys
    # Hint: return a hardcoded question and answer for the given topic/difficulty
    pass


def explain_concept(concept: str, level: str) -> dict:
    """Explain a concept at a specified comprehension level.

    Args:
        concept: The concept to explain (e.g. "mitosis", "supply and demand").
        level: Target level — "beginner", "intermediate", or "advanced".

    Returns:
        dict: status and explanation or error message.
    """
    # TODO: implement — return a dict with "status" and "report" keys
    # Hint: return a short explanation tailored to the requested level
    pass

## TODO 2: Define your agent

In [ ]:
agent = Agent(
    model="gemini-3-flash-preview",
    name="study_buddy",
    # TODO: Write an instruction that gives the agent an encouraging tutor persona.
    # Hint: describe how it should quiz students, when to give hints vs answers, etc.
    instruction="""You are a helpful study buddy.

Replace this with your agent's persona and guidelines.
""",
    tools=[create_flashcard, quiz_me, explain_concept],
)

## Run your agent

Once you've filled in the TODOs above, run this cell to test your agent!

In [ ]:
runner = InMemoryRunner(agent=agent, app_name="study_buddy")
runner.auto_create_session = True
user_content = types.Content(role="user", parts=[types.Part.from_text(text="I'm studying biology. Can you quiz me on photosynthesis and then create a flashcard for the key equation?")])

final_text = ""
async for event in runner.run_async(user_id="user1", session_id="session1", new_message=user_content):
    if event.content and event.content.parts:
        for part in event.content.parts:
            if part.text:
                final_text = part.text

print(final_text)

## Tips

- **Docstrings matter** — the ADK uses them as tool descriptions. Be specific about what each parameter means.
- **Fake data is fine** — the goal is designing the agent's interface, not building a real quiz engine.
- **Test edge cases** — what happens when a user asks about a topic your tools don't cover?

### Stretch Goals
- Add a `track_progress` tool that records quiz scores over time
- Add a `suggest_study_plan` tool that recommends what to review next
- Support multiple-choice vs. open-ended question formats